In [1]:
import pandas as pd
df = pd.read_json("/data/fine_tun_data2.json")
df.tail()

,instruction,input,output
1604,Categorize the input as advertisement or not a...,የምእራባውያን ማእቀብ አይበግረንም ፑቲን የምእራባውያን ማእቀብ ወደ ፊት ...,not advertisement
1605,Categorize the input as advertisement or not a...,የሶስትዮሽ ድርድሩ ዛሬ በአዲስ አበባ ተጀምሯል። የታላቁ ሕዳሴ ግድብ የኢ...,not advertisement
1606,Categorize the input as advertisement or not a...,ከሞጆ ከተማ በውስጥ የደረሰኝ መልእክት ነው።ስለመረጃው ትክክለኝነት አረጋ...,not advertisement
1607,Categorize the input as advertisement or not a...,Update ከአዲስ አበባ ከተማ አስተዳደር ፐብሊክ ሰርቪስ እና የሰው ሀብ...,not advertisement
1608,Categorize the input as advertisement or not a...,ጅግጅጋ ገብተዋል‼ ✔የሶማሌላንድ ፕሬዜዳንት ሙሴ ባሂ አብዲ ልኡካን ቡድና...,not advertisement


In [9]:
df[df['output']=='not advertisement'].iloc[1]['input']

'ለኢሬቻ በኣል ወደቢሾፍቱ የተጓዙት የሲዳማ ወጣቶች(ኤጄቶዎች) ከሰኣታት በፊት ቢሾፍቱ ገብተዋል። @tsegabwolde @tikvahethiopia'

In [ ]:
'Identify whether the given text is an advertisement or not advertisement from the given input. Make sure you respond only with advertisment or not advertisment. NOTHING ELSE. Input: ለኢሬቻ በኣል ወደቢሾፍቱ የተጓዙት የሲዳማ ወጣቶች(ኤጄቶዎች) ከሰኣታት በፊት ቢሾፍቱ ገብተዋል። @tsegabwolde @tikvahethiopia''

In [5]:
import pandas as pd
df = pd.read_csv("/data/wasu_mohammed_labeled.csv")
df.head()

,Unnamed: 0,channel_id,message_id,text,label,email
0,0,1490216693,8,መሰረት ተደርጎ ጥቃት ተፈፅሞብኛል በሚል በሀሰት የሽንት መሽኛ በፋሻና ፕ...,NaN,g2@trainee.com
1,1,1490216693,11,ኮምቦልቻ🔝 የጉዞ አድዋ ተጓዦች ኮምቦልቻ ገብተዋል። ሰላም እንደሆነ እና ...,NaN,g2@trainee.com
2,2,1490216693,13,‼️ በደቡብ ክልል በቴፒ ከተማ አደባባይ በወጡ ወጣቶች ላይ ፀጥታ አስከባ...,NaN,g2@trainee.com
3,3,1490216693,14,‼️ ከሁለት ወራት በፊት የሞትን ብርቱ ክንድ አሸንፈው ተነሱ የተባሉት አ...,NaN,g2@trainee.com
4,4,1490216693,15,ቴላቪቭ‼️ በሺህዎች የሚቆጠሩ ቤተ እስራኤላውያን አንድ በእስራኤል ፖሊስ ...,NaN,g2@trainee.com


In [8]:
df.shape

(16051, 6)

In [6]:
df['label'] = df['label'].fillna("Not Advertisement")
df.tail(5)

,Unnamed: 0,channel_id,message_id,text,label,email
16046,16046,1490216693,26113,አንዲት ኢትዮጵያዊት በዱባይ የ1 ሚሊዮን ዶላር ባለእድል ሆናለች ተብሏል።...,Not Avertisment,g2@trainee.com
16047,16047,1490216693,26114,ADVERTISMENT 🏑 መልካም ገና 🏑 🔜 💥ለ3 ቀናት ብቻ የሚቆይ ታላቅ...,financial service,g2@trainee.com
16048,16048,1490216693,26115,ድሬዎች🙏 ድሬ እውነትም የፍቅር ከተማ ሳምንቱን ሙሉ በውስጥሽ እንግዳ ሆኘ...,Not Avertisment,g2@trainee.com
16049,16049,1490216693,26116,ADVERTISMENT አስቸኳይ ሽያጭ…በልዩ ቅናሽ እነዚህን ጥራት ያላቸው ...,retail,g2@trainee.com
16050,16050,1490216693,26117,የአፍሪካ ህብረት ኮሚሽን ሊቀመንበር ሙሳ ፋኪ ማሃማት በኢትዮጵያ እና በሶ...,Not Avertisment,g2@trainee.com


In [ ]:
df

In [3]:
from datasets import Dataset

# Create a dictionary containing your Amharic text data
data_dict = {"text": df['text'].tolist()}

# Create a Dataset object
dataset = Dataset.from_dict(data_dict)


In [5]:
dataset.save_to_disk("../data/train")

Saving the dataset (0/1 shards):   0%|          | 0/5357 [00:00<?, ? examples/s]

In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the GNU General Public License version 3.

from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaConfig

# Function to load the main model for text generation
def load_model(model_name, quantization):
    model = LlamaForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        load_in_8bit=quantization,
        device_map="auto",
        low_cpu_mem_usage=True,
    )
    return model


# Function to load the PeftModel for performance optimization
def load_peft_model(model, peft_model):
    peft_model = PeftModel.from_pretrained(model, peft_model)
    return peft_model

# Loading the model from config to load FSDP checkpoints into that
def load_llama_from_config(config_path):
    model_config = LlamaConfig.from_pretrained(config_path) 
    model = LlamaForCausalLM(config=model_config)
    return model

In [1]:
from transformers import LlamaTokenizer


MAIN_PATH = '/model/Llama-2-7b-hf'
tokenizer = LlamaTokenizer.from_pretrained(MAIN_PATH)

example = 'አፖሎ ካለ " ኢንተርኔት ተቋርጦ ገንዘብ መላክም መቀበልም አልቻልኩ" ማለት የለም። *685# በመደወል አገልግሎቱን ይጠቀሙ።'


# garri_tokenizer = AutoTokenizer.from_pretrained("__the tokenizer__ you guys used")
tokens = tokenizer.tokenize(example)
print(tokens)

['▁አ', 'ፖ', 'ሎ', '▁ካለ', '▁"', '▁ኢንተርኔት', '▁ተቋርጦ', '▁ገንዘብ', '▁መላክ', 'ም', '▁መቀበል', 'ም', '▁አልቻል', 'ኩ', '"', '▁ማለት', '▁የለም።', '▁*', '6', '85', '#', '▁በመደ', 'ወል', '▁አገልግሎቱን', '▁ይጠቀሙ', '።']


In [3]:
print(len(tokenizer))

51008


In [4]:
example = 'አፖሎ ካለ " ኢንተርኔት ተቋርጦ ገንዘብ መላክም መቀበልም አልቻልኩ" ማለት የለም። *685# በመደወል አገልግሎቱን ይጠቀሙ።'


tokens = tokenizer.tokenize(example)
print(tokens)

['▁አ', 'ፖ', 'ሎ', '▁ካለ', '▁"', '▁ኢንተርኔት', '▁ተቋርጦ', '▁ገንዘብ', '▁መላክ', 'ም', '▁መቀበል', 'ም', '▁አልቻል', 'ኩ', '"', '▁ማለት', '▁የለም።', '▁*', '6', '85', '#', '▁በመደ', 'ወል', '▁አገልግሎቱን', '▁ይጠቀሙ', '።']


In [9]:
df['text'][0]

'መሰረት ተደርጎ ጥቃት ተፈፅሞብኛል በሚል በሀሰት የሽንት መሽኛ በፋሻና ፕላስተር በመጠቅለል ግጭት ለመቀስቀስ የሞከረው ግለስብ ፖሊስ በቁጥጥር ስር ዋለ። ተጠርጣሪ ምንም አይነት ጉዳት ሳይደርስበት ሆን ብሎ ብሄሬን መሰረት አድርገው በቢላዋ የሽንት መሽኛ ብልቴን ቆርጠው ጉዳት አድርሰውብኛል በሚል ድርጊቱን ያልፈፀሙ ግለሰቦችን ስም በመጥቀስ ለፖሊስ አቤቱታ ያቀረበው። ፖሊስ የቀረበውን አቤቱታ ተቀብሎ ሁኔታውን ለማጣራት ተከሳሽን ወደ ህክምና የላከ ሲሆን በተደረገ የህክምና ምርመራ ግለሰቡ ላይ ምንም አይነት ጥቃት ማረጋገጥ መቻሉን ተናግሯል። አሁን ላይ ስር ውሎ ምርመራው እየተጣራ መሆኑን ፖሊስ ጨምሮ ገልፆ ማንኛውም ሰው በዜጎች መካከል ግጭቶችና አለመግባባቶች እንዲፈጠሩ ከሚያደርጉ ተግባራት ሊቆጠብና ሊርቅ እንደሚገባ አሳስቧል:: ምንጭ፦ ድሬ ፖሊስ'

In [12]:
df = pd.read_csv("/data/wasu_mohammed_labeled.csv")

# Initialize a variable to accumulate the total word count
total_word_count = 0
total_tokens = 0
# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Get the text from the specified column
    text = row['text']
    if not isinstance(text, str): 
        continue

    
    # Count the number of words in the text
    word_count = len(text.split())

    # Accumulate the word count
    total_word_count += word_count
    tokens = tokenizer.tokenize(text)
    total_tokens+=tokens
    print(tokens)
    


# Display the total word count
print("Total Word Count:", total_word_count)
print("Total tokens count: ",total_tokens)

TypeError: unsupported operand type(s) for +=: 'int' and 'list'

In [13]:
total_tokens

0

In [11]:
df.shape

(16051, 6)

In [9]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    LlamaForCausalLM, 
    LlamaTokenizer,
    logging,
)
from peft import (
    LoraConfig,
    TaskType,
    prepare_model_for_int8_training,
    PeftModel
)
import torch


In [12]:
LLAMA_DIR = '/model/Llama-2-7b-hf'
tokenizer = LlamaTokenizer.from_pretrained(LLAMA_DIR)

model = LlamaForCausalLM.from_pretrained(LLAMA_DIR, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)
embedding_size = model.get_input_embeddings().weight.shape[0]

if len(tokenizer) != embedding_size:
    print("resize the embedding size by the size of the tokenizer")
    model.resize_token_embeddings(len(tokenizer))


new_model ='/home/abdulhamid_mussa/LLM_Finetuning_For_Amharic_Ad_Generation/output'
model = PeftModel.from_pretrained(model, new_model)

# Run text generation pipeline with our next model
prompt = "Who is Leonardo Da Vinci?"
pipe = pipeline(task="text-generation", model=model, tokenizer=model, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

resize the embedding size by the size of the tokenizer


The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

TypeError: LlamaForCausalLM.forward() got an unexpected keyword argument 'return_tensors'

: 

In [ ]:
ቨርቹዋል ረዳቶች እንደ Amazon&#39;s Alexa፣ Apple&#39;s Siri እና Google Assistant ያሉ የተለያዩ ባህሪያት እና ችሎታዎች አሏቸው። ነገር ግን በአጠቃላይ፣ ምናባዊ ረዳቶች እንደ የግል ረዳቶች ሆነው ያገለግላሉ፣ ለተጠቃሚዎች እንደ አስታዋሾች፣ ሙዚቃ መጫወት፣ የአየር ሁኔታ ዝመናዎች እና ጥያቄዎችን መመለስ ያሉ ተግባራትን ያከናውናሉ። ምናባዊ ረዳቶች እንደ የግል ረዳቶች ሆነው ያገለግላሉ፣ ለተጠቃሚዎች እንደ አስታዋሾች፣ ሙዚቃ መጫወት፣ የአየር ሁኔታ ዝመናዎች እና ጥያቄዎችን መመለስ ያሉ ተግባራትን ያከናውናሉ። ምናባዊ ረዳቶች እንደ የግል ረዳቶች ሆነው ያገለግላሉ፣ ለተጠቃሚዎች እንደ አስታዋሾች፣ ሙዚቃ መጫወት፣ የአየር ሁኔታ ዝመናዎች እና ጥያቄዎችን መመለስ ያሉ ተግባራትን ያከናውናሉ።